In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

# Задаём сиид для повторяемости
seed = 21

from keras.datasets import cifar10

# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# Разбиваем на катитгории
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))

model.add(Dropout(0.2))

model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(class_num))
model.add(Activation('softmax'))

epochs = 1
optimizer = 'adam'

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#print(model.summary())

#numpy.random.seed(seed)
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

In [2]:
import tensorflow as tf
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.001)

In [3]:
def grad_by_one_sample(model, x, y):
  with tf.GradientTape() as tape:
    pred = model(x[None, :])
    loss_by_one_sample = (y[None, :] - pred)**2
    grads = tape.gradient(loss_by_one_sample, model.trainable_variables)
  return grads

In [4]:
grads_list = []

In [5]:
def perform_one_epoch(model, opt, X_train, y_train, X_test, y_test, batch_size):

  for i_start, i_stop in zip(range(0, len(X_train) - batch_size, batch_size),
                             range(batch_size, len(X_train), batch_size)):
    
    def lambda_grad(var):
      return grad_by_one_sample(model, *var)
    l_grads = list(map(lambda_grad,zip(X_train[i_start: i_stop],
                                           y_train[i_start: i_stop])))
    grads = [tf.zeros_like(g) for g in l_grads[0]]
    for i in range(len(grads)):
      for j in range(len(l_grads)):
        grads[i] += l_grads[j][i]
    for i in range(len(grads)):
      grads[i] = grads[i] / len(l_grads)

    grads_list.append(grads)
    opt.apply_gradients(zip(grads, model.trainable_variables))

  test_pred = model(X_test)
  test_loss = tf.reduce_mean((y_test - test_pred)**2)
  return test_loss

In [6]:
def train_model(model,
                opt,
                X_train,
                y_train,
                X_test,
                y_test,
                batch_size,
                epoch_num = 10):
  for epoch in range(epoch_num):
    test_loss = perform_one_epoch(model, opt, X_train, y_train, X_test, y_test, batch_size)
    print(f'epoch: {epoch}, test_loss: {test_loss}')

In [7]:
# Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 9.29%


In [8]:
we = model.get_weights()

In [9]:
train_model(model,
            opt,
            X_train[:1000],
            y_train[:1000],
            X_test[:1000],
            y_test[:1000],
            batch_size=16,
            epoch_num=10)

epoch: 0, test_loss: 0.08664632588624954
epoch: 1, test_loss: 0.08178243786096573
epoch: 2, test_loss: 0.07932576537132263
epoch: 3, test_loss: 0.08002076297998428
epoch: 4, test_loss: 0.08336077630519867
epoch: 5, test_loss: 0.08229123800992966
epoch: 6, test_loss: 0.09384994953870773
epoch: 7, test_loss: 0.09261314570903778
epoch: 8, test_loss: 0.08989842236042023
epoch: 9, test_loss: 0.0944046899676323


In [10]:
# Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 35.07%


In [11]:
model.set_weights(we)

In [12]:
# Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 9.29%


In [ ]:
for p in range(len(grads_list)):
    opt.apply_gradients(zip(grads_list[p], model.trainable_variables))

In [ ]:
# Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))